In [2]:
import pymatgen

In [3]:
import ase.db

In [4]:
import numpy as np

In [5]:
import collections as coll

In [6]:
!wget https://cmr.fysik.dtu.dk/_downloads/mp_gllbsc.db

--2017-07-01 00:29:55--  https://cmr.fysik.dtu.dk/_downloads/mp_gllbsc.db
Resolving cmr.fysik.dtu.dk (cmr.fysik.dtu.dk)... 130.225.86.24, 2001:878:200:2010:3::24
Connecting to cmr.fysik.dtu.dk (cmr.fysik.dtu.dk)|130.225.86.24|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4542464 (4.3M) [text/plain]
Saving to: ‘mp_gllbsc.db.9’

mp_gllbsc.db.9      100%[=====================>]   4.33M  1.33MB/s   in 4.0s   

2017-07-01 00:30:00 (1.09 MB/s) - ‘mp_gllbsc.db.9’ saved [4542464/4542464]



In [7]:
con = ase.db.connect('mp_gllbsc.db')

In [8]:
from mpcontribs.io.archieml.mpfile import MPFile  

In [9]:
s = """
title: GLLB-SC Bandgaps
references.AEM: Advanced Energy Materials, Juli 22, 2014
references.PRA: O. Gritsenko, R. van  Leeuwen, E. van Lenthe, E. J. Baerends, Phys. Rev. A 1995, 51, 1944.  
references.PRB: M. Kuisma, J. Ojanen, J. Enkovaara, T. T. Rantala, Phys. Rev. B 2010, 82, 115106.  

contributor : Technical University of Denmark
authors: Ivano E. Castelli, Falco Hueser, Mohnish Pandey, Hong Li, Kristian S. Thygesen, Brian Seger, Anubhav Jain, Kristin A. Persson, Gerbrand Ceder, and Karsten W. Jacobsen   
explanation: Bandgaps calculated using GLLB-SC potential by Gritsenko, van Leeuwen, van Lenthe, and Baerends (GLLB), adapted by Kuisma et al to account for solids (-SC). The Kohn-Sham gap most basically refers to the energy difference between the bottom of the conduction band and the top of the valence band. The Quasi-Particle gap takes into account the derivative discontinuity of the chemical potential and gives the bandgap as the difference between ionization potenital and electron affinity. 
"""

In [10]:
mpfile = MPFile.from_string(s)
count = 0
x=y=z=w= []
for row in con.select('mpid'):
    d = coll.OrderedDict([])
    d['Kohn-Sham_Bandgap'] = coll.OrderedDict([])
    d['Derivative_Discontinuity'] = coll.OrderedDict([]) 
    d['Quasi-Particle_Bandgap'] = coll.OrderedDict([])
    count = count + 1
    mpid = 'mp-' + str(row.mpid)
    d['Kohn-Sham_Bandgap']['Indirect'] = row.gllbsc_ind_gap - row.gllbsc_disc
    d['Kohn-Sham_Bandgap']['Direct'] = row.gllbsc_dir_gap - row.gllbsc_disc
    d['Derivative_Discontinuity'] = row.gllbsc_disc
    d['Quasi-Particle_Bandgap']['Indirect'] = row.gllbsc_ind_gap
    d['Quasi-Particle_Bandgap']['Direct'] = row.gllbsc_dir_gap
    mpfile.add_hierarchical_data(d, identifier=mpid) 
    if count == 10:
        break

In [11]:
import plotly
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
plotly.offline.init_notebook_mode(connected = True) 

x1 = [row.mpid for row in con.select('mpid')]
y1 = [row.gllbsc_disc for row in con.select('mpid')]
y2 = [row.gllbsc_ind_gap for row in con.select('mpid')]
y3 = [row.gllbsc_dir_gap for row in con.select('mpid')]
                 
trace_1 = go.Bar(x = x1[0:10], y = y1[0:10], name = 'derivative_discontinuity') 
trace_2 = go.Bar(x = x1[0:10], y = y2[0:10], name = 'quasi-particle_bandgap(indirect)')
trace_3 = go.Bar(x = x1[0:10], y = y3[0:10], name = 'quasi-particle_bandgap(direct)')


data = [trace_1, trace_2, trace_3]
layout = go.Layout(barmode='stack', xaxis= dict(title= 'mp-id', type= 'category'), yaxis= dict(title= 'Energy (eV)'))
fig = go.Figure(data=data, layout=layout) 
stacked_bar = iplot(fig, filename='stacked-bar')